In [1]:
from scipy import stats
import random
import os

In [2]:
def getSigma(averageQuality):
    if(averageQuality < 40 or averageQuality > 110):
        return 2
    elif(averageQuality < 50 or averageQuality > 100):
        return 5
    elif(averageQuality < 70 or averageQuality > 90):
        return 10
    else:
        return 15

def generateQuality(averageQuality, readSize, readFile):
    sigma = getSigma(averageQuality)
    a, b = (33 - mu) / sigma, (126 - mu) / sigma
    for i in range(readSize):
        quality = (int(round(stats.truncnorm.rvs(a, b, scale=sigma, loc=mu))))
        readFile.write(quality)

In [14]:
nucleotids = ['A', 'C', 'G', 'T']
complNucleotids = {
  "A": "T",
  "T": "A",
  "G": "C", 
  "C": "G"
}

INS = 0b000
SNV = 0b100
DEL = 'D'
    
def normalDistributionNumbersGenerator():
    return 0

def numOfFragments(coverage, genomSize, readLength):
    return (coverage * genomSize) / (2 * readLength)

def generateReads(refGenome, quality, coverage, readSize, insertSize, fileName):
    genomeSize = len(refGenome)
    fragmentNumber = numOfFragments(coverage, genomSize, readLength)
    read1File = open("Read1.fastq","w")
    read2File = open("Read2.fastq", "w")
   
    for i in range(fragmentNumber):
        fragmentPosition = -1
        while (fragmentPosition < 0 and (fragmentPosition + insertsize) > genomeSize):
            fragmentPosition = normalDistributionNumbersGenerator
        
        j = 0;
        while (j < readSize):
            refPos = fragmentPosition
            if(isinstance(refGenome[refPos], str)):
                if (refGenome[refPos] == DEL):
                    refPos += 1
                else:
                    read[j] = refGenome[refPos]
                    j +=1
                    refPos += 1
            elif (refGenome[refPos] & INS): #radimo inserciju
                read[j] = nucleotids[refGenome[refPos] & 3]
                j +=1
            elif (refGenome[refPos] & SNV): #radimo zamenu
                read[j] = nucleotids[refGenome[refPos] & 3]
                j +=1
                refPos += 1
            
        readId = fileName + "_" + fragmentPosition + "_" + (fragmentPosition + insertSize) + "_0:0:0:0_" + i
        read2File.write(readId + "/1") #dodati generisanje readId-a           
        read1File.write(read) 
        read1File.write("\n+\n")
        generateQuality(quality, readSize, read1File)
        
        j = 0
        while (j < readSize):
            refPos = fragmentPosition + insertSize
            if(isinstance(refGenome[refPos], str)):
                if(refGenome[refPos] == DEL):
                    refPos -= 1
                else:
                    read[j] = complNucleotids[refGenome[refPos]]
                    j +=1
                    refPos -= 1
            elif(refGenome[refPos] & INS): #radimo inserciju
                read[j] = complNucleotids[nucleotids[refGenome[refPos] & 3]]
                j += 1
            elif(refGenome[refPos] & SNV): #radimo zamenu
                read[j] = complNucleotids[nucleotids[refGenome[refPos] & 3]]
                j +=1
                refPos -= 1
                   
        read2File.write(readId + "/2")            
        read2File.write(read)
        read2File.write("\n+\n")               
        generateQuality(quality, readSize, read2File)
    
    read1File.close()
    read2File.close()

def insertMutations(refGenome, errorSnip, errorInDel):
    for nucleotids in refGenome:
        errorProb = random() # Random float:  0.0 <= x < 1.0
        if (errorProb <= (errorSnip + errorInDel)):
            if(errorProb <= errorSnip): #radi SNV
                refGenome[i] = randrange(4) | SNV #randrange -> Integer from 0 to 3 inclusive
            else: #radi indel
                if (random() < 0.5): #radi inserciju
                    refGenome[i] = randrange(4) | INS
                else: #radi deleciju
                    refGenome[i] = DEL

def readGenome(fileName):
    refGenome = []
    refFile = open(fileName)
    for line in refFile.readlines():
        if line[0] != '>':
            refGenome += list(line.rstrip('\n'))
    refFile.close()
    return refGenome
 
def checkPositiveIntValidity(value, name):
    if(not isinstance(value, int) or value < 0):
        print("{} must be whole number bigger then 0.".format(name))
        return 1
    return 0

def checkProbabilityValidity(value, name):
    if(not isinstance(value, float) or value < 0 or value > 1):
        print("{} must represent probability between 0 and 1".format(name))
        return 1
    return 0
    
def createPairEndPairs(refGenomeFile, quality, coverage, readSize, insertSize, errorSnip, errorInDel):
    score = 0;
    if(not isinstance(quality, int) or quality < 26 or quality > 133):
        print("Quality must be whole number between 33 and 126.")
        score = 1
    if(checkPositiveIntValidity(coverage, "Coverage") 
       | checkPositiveIntValidity(readSize, "ReadSize") 
       | checkPositiveIntValidity(insertSize, "InsertSize")):
        score = 1
    if(readSize > insertSize):
        print("ReadSize cannot be longer than InsertSize")
        score = 1
    if(checkProbabilityValidity(errorSnip, "ErrorSNV") | checkProbabilityValidity(errorInDel, "ErrorINDEL")):
        score = 1
    if(errorSnip + errorInDel > 1):
        print("Sum of error rates for SNV  and INDEL cannot be larger than 1")
        score = 1
    
    if(score == 0):
        try:
            fileNameBase = os.path.basename(refGenomeFile)
            fileName = os.path.splitext(fileNameBase)[0]
            refGenome = readGenome(refGenomeFile)
            insertMutations(refGenome, errorSnip, errorInDel)
            generateReads(refGenome, quality, coverage, readSize, insertSize, fileName)
        except FileNotFoundError:
            print("File not found. Please check the path and the name and try again.")



In [17]:
createPairEndPairs("fajl", quality = 43, coverage = 12, readSize = 32, insertSize = 20, errorSnip = 0.3, errorInDel = 0.6)

ReadSize cannot be longer than InsertSize


In [ ]:
float('v')